In [1]:
import numpy as np
import galois

In [2]:
with open("challenge.png.encrypt", "rb") as f:
    c = f.read()

Known plaintext: we know the PNG magic number.

In [3]:
magic = bytes.fromhex("89504E470D0A1A0A")

In [4]:
def xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

In [5]:
bitstream = xor(magic, c).hex()
bitstream

'cafe6f6b4d3979ca'

In [6]:
seq = list(map(int, tuple("{0:08b}".format(int(bitstream, 16)))))
print(seq)

[1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0]


In [7]:
GF = galois.GF(2)
y = GF(seq)
fibo_lfsr = galois.berlekamp_massey(y, output="fibonacci")  # galois also works
print(fibo_lfsr)

Fibonacci LFSR:
  field: GF(2)
  feedback_poly: x^16 + x^11 + x^3 + x + 1
  characteristic_poly: x^16 + x^15 + x^13 + x^5 + 1
  taps: [1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1]
  order: 16
  state: [0 1 1 1 1 1 1 1 0 1 0 1 0 0 1 1]
  initial_state: [0 1 1 1 1 1 1 1 0 1 0 1 0 0 1 1]


Checking we get the magic number back...

89 50 4E 47 0D 0A 1A 0A

In [8]:
N = 8 * len(magic)
initial_state = seq[:16][::-1]
fibo_lfsr.reset(GF(initial_state))
keystream = np.packbits(fibo_lfsr.step(N).base)
for i in range(len(keystream)):
    print(hex(keystream[i] ^ c[i]))

0x89
0x50
0x4e
0x47
0xd
0xa
0x1a
0xa


Yay!

Reset LFSR and generate keystream with same length as cyphertext

In [9]:
N = 8 * len(c)
fibo_lfsr.reset(GF(initial_state))
keystream = np.packbits(fibo_lfsr.step(N).base)

Write the plain PNG byte by byte

In [10]:
with open("plain.png", "wb") as f:
    for i in range(len(keystream)):
        f.write(int(keystream[i] ^ c[i]).to_bytes())